# Upload Preliminaries

In [11]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import numpy as np
import networkx as nx
import torch
#import torch_geometric
import torch.nn as nn
import torch.nn.functional as F
#from torch_geometric.nn import GATConv
import random

In [12]:
# Tensorboard
from torch.utils.tensorboard import SummaryWriter

In [13]:
from dgl.data import CoraGraphDataset
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def load_cora_data():
    dataset = CoraGraphDataset()
    num_classes = dataset.num_classes
    g = dataset[0]
    scaler = MinMaxScaler()
    features = torch.tensor(scaler.fit_transform(g.ndata["feat"]), dtype=torch.float)
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    test_mask = g.ndata["test_mask"]
    return g, features, labels, num_classes, train_mask, test_mask

In [14]:
def create_line_graphs(G, t_levels):
  line_graphs = []
  features = []
  B = []
  current_g = G
  num_feats = G.ndata['feat'].shape[1]
  line_graphs.append(G)
  features.append(G.ndata['feat'])
  B.append(g.inc('in').to_dense())
  for t in range(1, t_levels):
    lg = current_g.line_graph(backtracking=True)
    line_graphs.append(lg)
    features.append(torch.zeros((lg.num_nodes(), num_feats)))
    B.append(lg.inc('in').to_dense())
    current_g = lg
  return line_graphs, features, B

In [15]:
def evaluate(train_logits, model, g, features, labels, train_mask, test_mask):
    tr_logits = train_logits[train_mask]
    tr_label = labels[train_mask]
    _, indices = torch.max(train_logits, dim=1)
    correct = torch.sum(indices == labels)
    train_acc = correct.item() * 1.0 / len(labels)
    model.eval()
    with torch.no_grad():
        test_logits = model(g, features)
        test_logits = test_logits[test_mask]
        labels = labels[test_mask]
        _, indices = torch.max(test_logits, dim=1)
        correct = torch.sum(indices == labels)
        test_acc = correct.item() * 1.0 / len(labels)
    return train_acc, test_acc

In [16]:
g, features, labels, num_classes, train_mask, test_mask = load_cora_data()

line_graphs_list, features_list, Incident_Matrix = create_line_graphs(g, 2)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


# Design the network

In [17]:
from dgl.nn import GraphConv

class Net2(nn.Module):
  def __init__(self, in_feats, out_feats):
    super(Net2, self).__init__()
    self.fusion = nn.Linear(in_feats, out_feats)
  def forward(self, h_t, B, t):
    B_star = B[0]
    for matrix in B[1:t]:
      B_star = torch.matmul(B_star, matrix)
    mask = B_star != 0
    B_star[mask] = 1
    b_t = torch.matmul(B_star, h_t)
    return b_t

In [18]:
from dgl.nn import GraphConv


class HGNNLayer(nn.Module):
  def __init__(self, in_feats, out_feats):
    super(HGNNLayer, self).__init__()
    self.conv = GraphConv(in_feats, out_feats)
    self.fuse = nn.Linear(out_feats, out_feats)

  def forward(self, g_t, f_t, B, prev_f, t):
    h_t = self.conv(g_t, f_t)
    B_T = torch.transpose(B, 0, 1)
    a_t = torch.mm(B_T, prev)
    a_t = self.fuse(a_t)
    h_t = h_t + a_t
    return F.relu(h_t)



In [19]:
class HGNN(nn.Module):
  def __init__(self, in_feats, h_feats):
    super(HGNN, self).__init__()
    self.conv = GraphConv(in_feats, h_feats)
    self.hgnn = HGNNLayer(in_feats, h_feats)
    self.net2 = Net2(h_feats, h_feats)
    self.combine = nn.Linear(h_feats, h_feats)

  def forward(self, G, f, B):
    h = []
    embeddings = []

    h_0 = F.relu(self.conv(G[0], f[0]))
    h.append(h_0)
    embeddings.append(h_0)

    for t in range(1, len(G)):
      h_t = self.hgnn(G[t], f[t], B[t-1], f[t-1], t)
      h.append(h_t)
      embeddings.append(self.net2(h_t, B, t))

    concatenated_embeddings = torch.cat(embeddings, dim=0)

    f_star = torch.mean(torch.stack(embeddings), dim=0)

    h[0] = f_star

    return h


In [20]:
class StackLayers(nn.Module):
  def __init__(self, in_feats, num_classes):
    super(StackLayers, self).__init__()
    self.hidden_layer1 = HGNN(in_feats, 16)
    self.output_layer = HGNN(16, num_classes)

  def forward(self, g, f, B):
    h1 = self.hidden_layer1(g, f, B)
    z = self.output_layer(g, h1, B)
    return z[0]


# Train and Evaluate the Model

In [ ]:
model = StackLayers(features.shape[1], num_classes)

print("the model")
print(model)


In [24]:
import time
import numpy as np


# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
dur = []
for epoch in range(100):
    if epoch >= 3:
        t0 = time.time()
    optimizer.zero_grad()


    model.train()
    logits = model(line_graphs_list, features_list, Incident_Matrix)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    torch.autograd.set_detect_anomaly(True)
    loss.backward()
    optimizer.step()


    if epoch >= 3:
        dur.append(time.time() - t0)

    train_acc, test_acc = evaluate(logits, model, g, features, labels, train_mask, test_mask)
    print(
        "Epoch {:05d} | Loss {:.4f} | Train Acc {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), train_acc, test_acc, np.mean(dur)
        )
    )

h_t torch.Size([10556, 16])
prev, torch.Size([2708, 16])
B_T torch.Size([10556, 2708])
final h_t torch.Size([10556, 16])
t is  1
B*:  torch.Size([2708, 10556])
b_t:  torch.Size([2708, 16])
h_t torch.Size([10556, 7])
prev, torch.Size([2708, 7])
B_T torch.Size([10556, 2708])
final h_t torch.Size([10556, 7])
t is  1
B*:  torch.Size([2708, 10556])
b_t:  torch.Size([2708, 7])


c:\Users\solma\anaconda3\Lib\site-packages\torch\autograd\__init__.py:266: UserWarning: Error detected in MmBackward0. Traceback of forward call that caused the error:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\solma\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\solma\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\solma\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "c:\Users\solma\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "c:\Users\solma\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever
    self._run_once()
  File "c:\Users\solma\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once
    handle._run()
  F

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [10556, 2708]] is at version 4; expected version 3 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!